## Imports

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.sparse.linalg import svds
from numpy.linalg import norm, svd

import itertools
import math
import re

#from bs4 import BeautifulSoup
#from nltk.corpus import stopwords
#from gensim.models import word2vec, Word2Vec

from util.word2vec_as_MF import Word2vecMF
from util.functions import *

import time
dimension = 100

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read and preprocess enwik9

In [21]:
real_sentences = load('data/x1')

Parsing sentences from training set


# Gensim

In [22]:
'''%%time
skip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'''

'%%time\nskip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'

In [23]:
'''skip.get_latest_training_loss()'''

'skip.get_latest_training_loss()'

## Train ro_sgns model starting from SVD of SPPMI

In [24]:
# Create word2vec as matrix factorization model
model_enwik = Word2vecMF()
#model_enwik.data_to_matrices(real_sentences, dimension, 5, 'enwik-200/matrices.npz')
model_enwik.data_to_matrices(real_sentences, dimension, 5, False)

In [25]:
# If the model has been already created, load it from file
'''model_enwik = Word2vecMF()
model_enwik.load_matrices(from_file='enwik-200/matrices1.npz')'''

"model_enwik = Word2vecMF()\nmodel_enwik.load_matrices(from_file='enwik-200/matrices1.npz')"

In [26]:
'''# SVD initialization
SPPMI = np.maximum(np.log(model_enwik.D) - np.log(model_enwik.B), 0)
# print SPPMI
u, s, vt = svds(SPPMI, k=dimension)
C_svd = u.dot(np.sqrt(np.diag(s))).T
W_svd = np.sqrt(np.diag(s)).dot(vt)
print(model_enwik.MF(C_svd, W_svd))'''

'# SVD initialization\nSPPMI = np.maximum(np.log(model_enwik.D) - np.log(model_enwik.B), 0)\n# print SPPMI\nu, s, vt = svds(SPPMI, k=dimension)\nC_svd = u.dot(np.sqrt(np.diag(s))).T\nW_svd = np.sqrt(np.diag(s)).dot(vt)\nprint(model_enwik.MF(C_svd, W_svd))'

In [27]:
'''model_enwik.C = C_svd
model_enwik.W = W_svd
#model_enwik.save_CW('enwik-200/initializations/SVD_dim200', 0)
print(model_enwik.C.shape, model_enwik.W.shape, model_enwik.B.shape, model_enwik.D.shape)'''

"model_enwik.C = C_svd\nmodel_enwik.W = W_svd\n#model_enwik.save_CW('enwik-200/initializations/SVD_dim200', 0)\nprint(model_enwik.C.shape, model_enwik.W.shape, model_enwik.B.shape, model_enwik.D.shape)"

In [28]:
X0 = 1/norm((model_enwik.B+model_enwik.D)/4, 'fro')*model_enwik.grad_MF(
    np.zeros([dimension,model_enwik.B.shape[0]]), np.zeros([dimension,model_enwik.B.shape[0]]))
print(X0.shape)
u, s, vt = svds(X0, k=dimension)
C0 = u.dot(np.sqrt(np.diag(s))).T
W0 = np.sqrt(np.diag(s)).dot(vt)
print(model_enwik.MF(C0, W0))

(356, 356)
(1783763.7762035725, 5.251327505899099e-14)


In [29]:
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='PS', 
               d=dimension,
               eta = 5e-5,
               MAX_ITER=10,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

'# Train the model\nstart_time = time.time()\nopt_experiment(model_enwik,\n               mode=\'PS\', \n               d=dimension,\n               eta = 5e-5,\n               MAX_ITER=10,\n               from_iter=0,\n               start_from=\'SVD\',\n               init=(True, C_svd, W_svd), display=True)\nprint("--- %s seconds ---" % (time.time() - start_time))'

In [ ]:
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='AM', 
               d=dimension,
               eta = 5e-6,
               lbd = 1.0,
               MAX_ITER=189000,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

'# Train the model\nstart_time = time.time()\nopt_experiment(model_enwik,\n               mode=\'AM\', \n               d=dimension,\n               eta = 5e-6,\n               lbd = 1.0,\n               MAX_ITER=189000,\n               from_iter=0,\n               start_from=\'SVD\',\n               init=(True, C_svd, W_svd), display=True)\nprint("--- %s seconds ---" % (time.time() - start_time))'

In [ ]:
start_time = time.time()
opt_experiment(model_enwik,
               mode='BFGD',
               d=dimension,
               eta = 1e-6,
               reg = 0.006,
               MAX_ITER=190000,
               from_iter=0,
               start_from='X0_full',
               init=(True, C0, W0), display=True)
print("--- %s seconds ---" % (time.time() - start_time))


enwik-200/BFGDiter_fromX0_full_dim100_step1e-06_0.006
Iter #: 100 loss (1621255.1016031264, 0.002798092004622574)
Iter #: 200 loss (1497570.5076319438, 0.0021177411148418234)
Iter #: 300 loss (1413499.8261292032, 0.0012984027934174372)
Iter #: 400 loss (1353973.041003951, 0.0014624949619139564)
Iter #: 500 loss (1308949.5045622494, 0.0033527840878211537)
Iter #: 600 loss (1273067.5570341928, 0.0058166326070082566)
Iter #: 700 loss (1243558.9755853657, 0.008469237218521582)
Iter #: 800 loss (1218823.103892192, 0.011084429714644125)
Iter #: 900 loss (1197833.2922820796, 0.013538762351705423)
Iter #: 1000 loss (1179911.979329155, 0.0157521841235404)
Iter #: 1100 loss (1164605.7055433504, 0.017677197516062934)
Iter #: 1200 loss (1151576.839455201, 0.019305652194758664)
Iter #: 1300 loss (1140526.6630331478, 0.020661943580257466)
Iter #: 1400 loss (1131166.5686767232, 0.021784891360438766)
Iter #: 1500 loss (1123223.559605428, 0.02271290841442468)
Iter #: 1600 loss (1116453.6342020906, 0.02

In [ ]:
'''model = Word2Vec(real_sentences, size=200, window=5, min_count=5, workers=4)
fname = 'original'
model.save(fname)
model = Word2Vec.load(fname)  # you can continue training with the loaded model!'''